# TITAN Tutorial

---

<img align="right" width="300" height="300" src="../images/hail_case_tracks.png">

This interactive tutorial takes you through the steps of how to run the Thunderstorm Identification, Tracking, Analysis and Nowcasting (TITAN) suite. TITAN was originally designed as an algorithm to objectively identify and track thunderstorms from weather radar data for a weather modification experiment in South Africa in the 1980s. Now, Titan includes forecasting, storm analysis, and climatological analysis. TITAN now refers to the larger system in which the original application is one component.

Titan is described in more detail in [Dixon and Wiener (1993)](https://doi.org/10.1175/1520-0426(1993)010%3C0785:TTITAA%3E2.0.CO;2).


---

## Tutorial Overview
### 1. Setup

#### Download raw data and prepare parameter files

Raw data files that are provided:
* A hail storm in Alberta, observed by the Strathmore radar 40 km east Calgary.
* A derecho event in Ontario, observed by the King City radar 40 km north of Toronto.

Both of these are 10 cm (S-band) Gematronik dual polarization radars.

The data (as a .tgz file) can be downloaded from Mike's Google drive at NCAR:

* [AMS Titan data](https://drive.google.com/drive/folders/1OzjLzsGhSBAKDvFzJeBLGHYW0RlYU0hZ?usp=sharing)

To be consistent, you should create a ```$HOME/data``` directory and copy the .tgz file there.

Then:

```
  tar xvfz ams2025_titan.raw.tgz
```

That will create the following tree:

```
  ~/data/ams2025/ERA5/20220521
  ~/data/ams2025/ERA5/20240806
  ~/data/ams2025/radar/raw/hail/20240806*.h5
  ~/data/ams2025/radar/raw/derecho/20220521*.h5
```

### 2. Output data

After the full analysis has been run, the following derived data directories should exist:

```
  ~/data/ams2025/ERA5/spdb/Strathmore/20240806* (soundings from ERA5)
  ~/data/ams2025/ERA5/spdb/KingCity/20220521* (soundings from ERA5)
  ~/data/ams2025/radar/cfradial/qc/Strathmore/20240806/cfrad.20240806*nc (cfradial after QC)
  ~/data/ams2025/radar/cfradial/qc/KingCity/20220521/cfrad.20220521*nc (cfradial after QC)
  ~/data/ams2025/radar/cfradial/pid/Strathmore/20240806/cfrad.20240806*nc (cfradial PID)
  ~/data/ams2025/radar/cfradial/pid/Strathmore/20240806/cfrad.20240806*nc (cfradial PID)
  ~/data/ams2025/radar/cart/qc/Strathmore/20240806/ncf_20240806*nc (Cartesian MDC CF-compliant netcdf)
  ~/data/ams2025/radar/cart/qc/KingCity/20220521/ncf_202205216*nc (Cartesian MDC CF-compliant netcdf)
  ~/data/ams2025/titan/storms/Strathmore/20240806* (Titan binary files)
  ~/data/ams2025/titan/storms/KingCity/20220521* (Titan binary files)
  ~/data/ams2025/titan/ascii/Tracks2Ascii.hail.txt (Titan output converted by Tracks2Ascii)
  ~/data/ams2025/titan/ascii/Tracks2Ascii.derecho.txt (Titan output converted by Tracks2Ascii)
  ~/data/ams2025/titan/netcdf/Strathmore/titan_20240806.nc (Titan output converted by Tstorms2NetCDF)
  ~/data/ams2025/titan/netcdf/KingCity/titan_20220521.nc (Titan output converted by Tstorms2NetCDF)
```

### 3. Note on task cells

This notebook uses two colored cells to indicate tasks.

<div class="alert alert-block alert-info"> <b>File Task: modify parameters in text files.</b> 

These text blocks help the user modify the parameter files or other functions in *external* text files.

</div>

<div class="alert alert-block alert-warning"> <b>Cell Task: run a command in Jupyter notebook cell.</b> 

These text blocks instruct the users to run a command *in* a cell within the Jupyter notebook. If you prefer, you are welcome to copy the commands (minus the ! symbol) into a terminal window.

</div>

---

# 1. Setup
## Environment and packages

First, we import the required python packages to run this notebook. Most of the LROSE processing can be done with the os package and shell commands.

In [ ]:
import os
import subprocess

## 1.1 Set up directories

We need to set up the required data directories. The raw radar data will be grabbed from the S3 buckt. We delete any existing files and directories specific to this tutorial to ensure we're starting with clean directories and files.

In [ ]:
# Check out the project repository
!mkdir -p ~/git
!cd ~/git
!git clone https://github.com/ncar/lrose-titan

The structure of the ams2025 part of this repo is as follows:

```
  ~/git/lrose-titan/projects/ams2025/params
  ~/git/lrose-titan/projects/ams2025/scripts
  ~/git/lrose-titan/projects/ams2025/color_scales
  ~/git/lrose-titan/projects/ams2025/maps
```

## 1.2 Set up the environment

In the scripts directory you will find the file:

```
  ~/git/lrose-titan/projects/ams2025/scripts/set_env_vars
```

This file setup up the environment, and is sourced by all of the scripts that we run for this project.

The default contents are as follows:

```
  setenv DATA_DIR $HOME/data/ams2025
  setenv PROJ_DIR $HOME/git/lrose-titan/projects/ams2025
```

The default settings work as is, if you have followed these instructions.

If you have a different layout, edit ```set_env_vars``` appropriately.

# 2. Prepare data for analysis

The following sections describe two quality control setps and how to run the scripts.

## 2.1: Option 1 - Apply quality control (QC) on the raw radar data and convert to CfRadial format using RadxConvert

In the hail case, there is no significant signal interference. 
![Alt text](../images/lrose/hail.dbz.no_qc.png)


In the derecho case, considerable interference is present, appearing as radial spikes.  
![Alt text](../images/lrose/derecho.dbz.no_qc.png)


Closer inspection of these spikes shows that the interference sources are not coherent with the radars, as indicated by:  

* Low SQI (NCP)  
* Moderately low SNR  

To address this, we use `RadxConvert` to censor data fields based on thresholds applied to the input fields. Specifically, data are removed at gates where **both** conditions are met:  

* SQI (NCP) < 0.2  
* SNR < 25 dB  

Since later QC steps require signal-to-noise ratio (SNR), the SNR field is derived from reflectivity (DBZ) and added during processing.  

Finally, the raw HDF5 files are converted to CfRadial format using `RadxConvert` with this simple quality control applied.  

<div class="alert alert-block alert-warning"> <b>Cell Task: Run QC on hail case data.</b> 
    <br>
    Run the hail case QC script:
    <br>
    <code lang="bash">!./run_RadxConvert.qc.hail</code>
</div>

In [ ]:
# Run QC on hail case data
!./run_RadxConvert.qc.hail

<div class="alert alert-block alert-warning"> <b>Cell Task: Run QC on derecho case data.</b> 
    <br>
    Run the derecho case QC script:
    <br>
    <code lang="bash">!./run_RadxConvert.qc.derecho</code>
</div>

In [ ]:
# Run QC on derecho case data
!./run_RadxConvert.qc.derecho

## 2.2: Option 2 - Computing PID as an alternative method of censoring using RadxPid

An alternative method for cleaning up interference is to run RadxPid, and censor non-meteorological echoes.

First, we have to download the ERA5 reanalysis for these cases, and we can use that to save the model-based soundings:

<div class="alert alert-block alert-warning"> <b>Cell Task: Run ERA5 sounding scripts.</b> 
    <br>
    Run the ERA5 sounding scripts:
    <br>
    <code lang="bash">!./run_Mdv2SoundingSpdb.ERA5.hail</code><br>
    <code lang="bash">!./run_Mdv2SoundingSpdb.ERA5.derecho</code>
</div>

In [ ]:
# Get the ERA5 sounding for hail case data
!./run_Mdv2SoundingSpdb.ERA5.hail

In [ ]:
# Get the ERA5 sounding for derecho case data
!./run_Mdv2SoundingSpdb.ERA5.derecho

And we can then run RadxPid:
<div class="alert alert-block alert-warning"> <b>Cell Task: Run PID on hail case data.</b> 
    <br>
    Run the hail case PID script:
    <br>
    <code lang="bash">!./run_RadxPid.hail</code>
</div>

In [ ]:
# Run PID on hail case data
!./run_RadxPid.hail

<div class="alert alert-block alert-warning"> <b>Cell Task: Run PID on derecho case data.</b> 
    <br>
    Run the derecho case PID script:
    <br>
    <code lang="bash">!./run_RadxPid.derecho</code>
</div>

In [ ]:
# Run PID on derecho case data
!./run_RadxPid.derecho

The following shows the PID field for the derecho case:

![Alt text](../images/lrose/derecho.pid.png)

The interference is identified as clutter in this case.

And the following shows the result of using PID to clean up the reflectivity field:

![Alt text](../images/lrose/derecho.dbz.censored_by_pid.png)

For this tutorial we will use the QC data created by RadxConvert.

## 2.3: Convert to Cartesian grid using Radx2Grid

Titan requires input data in Cartesian coordinates, rather than polar.
To perform this transformation, we run the following to convert the data to Cartesian grid format.

<div class="alert alert-block alert-warning"> <b>Cell Task: Convert hail case to Cartesian grid.</b> 
    <br>
    Run the hail case grid conversion script:
    <br>
    <code lang="bash">!./run_Radx2Grid.hail</code>
</div>

In [ ]:
# Convert hail case to Cartesian grid
!./run_Radx2Grid.hail

<div class="alert alert-block alert-warning"> <b>Cell Task: Convert derecho case to Cartesian grid.</b> 
    <br>
    Run the derecho case grid conversion script:
    <br>
    <code lang="bash">!./run_Radx2Grid.derecho</code>
</div>

In [ ]:
# Convert derecho case to Cartesian grid
!./run_Radx2Grid.derecho

# 3. Run TITAN storm tracking

Run the TITAN algorithm to identify and track storms.

Titan runs on the Cartesian gridded data, using the DBZ field and optionally the VEL field to compute storm rotation.


<div class="alert alert-block alert-warning"> <b>Cell Task: Run TITAN on hail case data.</b> 
    <br>
    Run the hail case TITAN script:
    <br>
    <code lang="bash">!./run_Titan.hail</code>
</div>

In [ ]:
# Run TITAN on hail case data
!./run_Titan.hail

In [ ]:
# Run PID on hail case data
!./run_RadxPid.hail

In [ ]:
# Run PID on hail case data
!./run_RadxPid.hail

In [ ]:
# Run PID on hail case data
!./run_RadxPid.hail

In [ ]:
# Run PID on hail case data
!./run_RadxPid.hail

In [ ]:
# Run PID on hail case data
!./run_RadxPid.hail

In [ ]:
# Run PID on hail case data
!./run_RadxPid.hail

<div class="alert alert-block alert-warning"> <b>Cell Task: Run TITAN on derecho case data.</b> 
    <br>
    Run the derecho case TITAN script:
    <br>
    <code lang="bash">!./run_Titan.derecho</code>
</div>

In [ ]:
# Run TITAN on derecho case data
!./run_Titan.derecho

# 4. Convert TITAN binary output to readable format

## 4.1 Convert TITAN binary output to ASCII format for analysis.

<div class="alert alert-block alert-warning"> <b>Cell Task: Convert hail case TITAN output to ASCII.</b> 
    <br>
    Run the hail case ASCII conversion script:
    <br>
    <code lang="bash">!./run_Tracks2Ascii.hail</code>
</div>

In [2]:
# Convert hail case TITAN output to ASCII
!./run_Tracks2Ascii.hail

zsh:1: no such file or directory: ./run_Tracks2Ascii.hail


<div class="alert alert-block alert-warning"> <b>Cell Task: Convert derecho case TITAN output to ASCII.</b> 
    <br>
    Run the derecho case ASCII conversion script:
    <br>
    <code lang="bash">!./run_Tracks2Ascii.derecho</code>
</div>

In [ ]:
# Convert derecho case TITAN output to ASCII
!./run_Tracks2Ascii.derecho

## 4.2: Convert TITAN output to NetCDF format

Convert TITAN binary output to NetCDF format for further analysis.

<div class="alert alert-block alert-warning"> <b>Cell Task: Convert hail case TITAN output to NetCDF.</b> 
    <br>
    Run the hail case NetCDF conversion script:
    <br>
    <code lang="bash">!./run_Tstorms2NetCDF.hail</code>
</div>

In [ ]:
# Convert hail case TITAN output to NetCDF
!./run_Tstorms2NetCDF.hail

<div class="alert alert-block alert-warning"> <b>Cell Task: Convert derecho case TITAN output to NetCDF.</b> 
    <br>
    Run the derecho case NetCDF conversion script:
    <br>
    <code lang="bash">!./run_Tstorms2NetCDF.derecho</code>
</div>

In [ ]:
# Convert derecho case TITAN output to NetCDF
!./run_Tstorms2NetCDF.derecho

# 5. Plot Output